In [1]:
%pip install google-generativeai

   ---------------------------------------- 0.0/760.0 kB ? eta -:--:--
   --------------------------- ------------ 524.3/760.0 kB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 760.0/760.0 kB 4.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.3 MB ? eta -:--:--
   ---------- ----------------------------- 3.1/12.3 MB 15.3 MB/s eta 0:00:01
   ----------------- ---------------------- 5.5/12.3 MB 16.0 MB/s eta 0:00:01
   ----------------------- ---------------- 7.1/12.3 MB 11.2 MB/s eta 0:00:01
   ----------------------- ---------------- 7.1/12.3 MB 11.2 MB/s eta 0:00:01
   ----------------------- ---------------- 7.1/12.3 MB 11.2 MB/s eta 0:00:01
   ----------------------- ---------------- 7.1/12.3 MB 11.2 MB/s eta 0:00:01
   ----------------------- ---------------- 7.1/12.3 MB 11.2 MB/s eta 0:00:01
   ----------------------- ---------------- 7.1/12.3 MB 11.2 MB/s eta 0:00:01
   ----------------------- ---------------- 7.1/12.3 MB 11.2 MB/s eta 0:00:

In [18]:
import os
import base64
import google.generativeai as genai
from dotenv import load_dotenv
from PIL import Image
from io import BytesIO
load_dotenv()

True

In [19]:
api_key = os.environ.get("GEMINI_API_KEY")
genai.configure(api_key=api_key)

# Initialize the model with specific configurations
model = genai.GenerativeModel(
    "gemini-1.5-flash-8b",
    generation_config=genai.GenerationConfig(temperature=0)
)

In [30]:
def predict_object_context(model, image, class_names):
    # Decode the base64 image string
    image_data = base64.b64decode(image)
    
    # Convert the binary data to an image using PIL
    image_context = Image.open(BytesIO(image_data))
    
    # Format the class names into a human-readable list format
    class_list = ', '.join(class_names)

    # Define the prompt with the image and class names
    prompt = f"""
    You are given an image and a list of class names, each representing a segmented object in the image. 
    Your task is to check whether each object, based on its segmentation, is placed on a blue cloth in the image. 
    The list of class names is: {class_list}.
    
    For each class name, return a JSON object where the key is the class name and the value is a boolean (True or False) 
    indicating if the object is on the blue cloth. Here is the format of the expected output:

    {{
    "class_1": True/False,
    "class_2": True/False,
    "class_3": True/False,
    ...
    }}

    Use the image content and segmentation to determine the color and position of the cloth and the objects. 
    Base the decision on whether the object's segmented area overlaps with the blue cloth area in the image.
    """

    # Generate the content using the model
    result = model.generate_content([image_context, "\n\n", prompt])

    return result.text

In [32]:
res = predict_object_context(model, "...", ["forceps", "gauze", "scissors"])

In [33]:
res

'```json\n{\n  "forceps": True,\n  "gauze": False,\n  "scissors": False\n}\n```'